In [1]:
import pandas as pd
import numpy as np
import random
import itertools
import math

# Técnicas de Amostragem

## Estratificada com escolha aleatória para os subgrupos

In [2]:
## Stratified Sampling combined with Random Sampling 

initial_num_processos = [1, 150, 18500, 22900, 39990]
identifiers = np.arange(1, 5)
choices = {}
for identifier, (current, next) in zip(identifiers, itertools.pairwise(initial_num_processos)):
    if identifier == identifiers[-1]:
        frame = np.arange(current, next)
    else:
        frame = np.arange(current, next-1)
    size = math.ceil(len(frame)*0.05)
    if size < 100: # tamanho minimo de 100
        size = 100
    choices[identifier] = ['A', np.random.choice(frame, size, replace=False)] # random sampling

# [len(choices[key]) for key in choices.keys()], choices

## Estratificada com escolha sistemática para os subgrupos

In [3]:
def systematic_sampling(df, first_number, last_number, step):
    indexes = np.arange(first_number, last_number, step=step)
    return indexes

In [4]:
## Stratified Sampling combined with Systematic Sampling
initial_num_processos = [1, 150, 18500, 22900, 39990]
identifiers = np.arange(1, 5)
choices_systematic = {}
for identifier, (current, next) in zip(identifiers, itertools.pairwise(initial_num_processos)):
    if identifier == identifiers[-1]:
        frame = np.arange(current, next + 1)
    else:
        frame = np.arange(current, next)
        
    size = math.ceil(len(frame)*0.05)

    if size < 100: # tamanho minimo de 100
        size = 100

    np.random.seed(0)
    starting_number = random.randint(current, current + 20) 
    
    # starting_number = current

    last_number = next
    interval = len(frame)//size
    choices_systematic[identifier] = ['B', systematic_sampling(frame, starting_number, last_number, interval)] # systematic sampling

# [len(choices[key]) for key in choices.keys()], choices

In [5]:
## Escrevendo resultado em uma planilha do Excel com macros ativadas

import openpyxl
import os
import string
import pandas as pd
import numpy as np

path = '.' 
target_file = 'filename.xlsm' 
sheetname = 'Planilha1'

filename = os.path.join(path, target_file)
if os.path.isfile(filename):    
    wb = openpyxl.load_workbook(filename, keep_vba=True)
    sheet = wb[sheetname]
else:
    wb = openpyxl.Workbook() 
    sheet = wb.create_sheet(sheetname)
    wb.save(filename)

# letra da coluna para número
d = dict()
for x, y in zip(range(0, 27), string.ascii_lowercase):
    d[x] = y.upper()


## Salvando resultado no Excel
Primeiro resultado do Amostragem estratificada com escolha aleatória e, embaixo, da Amostragem estratificada com escolha sistemática

In [6]:
# apagar as primeiras 2 colunas sem contar row 1 (header)
max_row = sheet.max_row
max_col = 2

for row in range(max_row):
    row += 1
    if row == 1: continue # pular headers
    for col in range(max_col):
        sheet['{}{}'.format(d[col], row)] = None
        col += 1

wb.save(filename)

In [7]:
# preencher as colunas sem contar header
df_stratified_random =pd.DataFrame.from_records(choices[1]).T.fillna(method="ffill")
df_stratified_systematic = pd.DataFrame.from_records(choices_systematic[1]).T.fillna(method="ffill")

df = pd.concat([df_stratified_random, df_stratified_systematic])

for row in range(len(df.index)):
    row += 1 # no excel numeracao começa com 1
    if row == 1: continue # pular headers
    for col in range(max_col): 
        value = df.iloc[row-2][col]
        sheet['{}{}'.format(d[col], row)] = value
        col += 1

wb.save(filename)